<font color="firebrick" size="6"><b>리뷰 개수 카운트 </b></font>


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import requests
from bs4 import BeautifulSoup

import time
import os

import json
import warnings
warnings.filterwarnings("ignore")

In [2]:
os.chdir("../../../smartstore") # signaturehelper module 호출하기 위해 경로 설정

In [3]:
os.getcwd()

'C:\\Users\\chosun\\Desktop\\smartstore'

In [4]:
pd.set_option('float_format', '{:.2f}'.format)

<font color="Maroon" size="5"><b>avg_sale 조건 설정하여 필터링</b></font>

In [4]:
pd.options.display.max_rows = 100  
data = pd.read_csv("conditionedData/data/4_avg_sale/4_avg_sale_requests.csv")
keywords = data["relative_keyword"]
data

,relative_keyword,total_search,total_store_count,competitive_rate,avg_price,avg_sale
0,앱코키보드,14530.0,40613.0,2.80,73669.0,548445.0
1,키즈경량패딩,28730.0,76995.0,2.68,76983.0,1002830.0
2,후쿠오카버스투어,22720.0,546.0,0.02,58505.0,7110358.0
3,식전영상,27600.0,432.0,0.02,33231.0,1019754.0
4,꾸셀베이스오리지널,2270.0,346.0,0.15,27540.0,6445.0
...,...,...,...,...,...,...
18222,EMIS가방,12380.0,7205.0,0.58,43951.0,645968.0
18223,나이키에코백,6410.0,1839.0,0.29,11203.0,116761.0
18224,아페쎄에코백,9960.0,12389.0,1.24,86765.0,731364.0
18225,이미스호보백,11450.0,1839.0,0.16,43885.0,786907.0


In [6]:
data.describe()

,total_search,total_store_count,competitive_rate,avg_price,avg_sale
count,18227.00,18227.00,18227.00,18227.00,18227.00
mean,5110.16,21428.29,8.05,31141.13,1574561.25
std,5386.21,26854.82,13.40,22083.36,6103682.35
min,1010.00,1.00,0.00,6002.00,100.00
25%,1660.00,1673.00,0.47,14532.00,183816.00
50%,2940.00,8824.00,2.30,23929.00,528041.00
75%,6160.00,32315.00,9.35,41616.50,1398348.00
max,29960.00,105909.00,102.15,99923.00,425049333.00


In [6]:
"""2Q 이하 제거 (대략 기준 1개월 매출 50만원 이상)"""
pd.options.display.max_rows = None  
standard = np.percentile(data["avg_sale"], 50)
new_data = data[data["avg_sale"] >= standard].sort_values("avg_sale")
new_data.index = np.arange(len(new_data["avg_price"]))
new_data

,relative_keyword,total_search,total_store_count,competitive_rate,avg_price,avg_sale
0,대파크림치즈,10330.0,1375.0,0.13,17500.0,528041.0
1,열반사단열재,5750.0,8235.0,1.43,48356.0,528275.0
2,맥세이프실리콘케이스,1470.0,92439.0,62.88,33779.0,528374.0
3,에어프라이어용기,2730.0,41153.0,15.07,12648.0,528417.0
4,커트보,2390.0,58518.0,24.48,11314.0,528460.0
...,...,...,...,...,...,...
9109,닥터로운셀라큐어,1590.0,18.0,0.01,34450.0,175149500.0
9110,셀라큐어,10750.0,107.0,0.01,47172.0,175149500.0
9111,셀라큐어문제성발톱셀라솔루션,2930.0,43.0,0.01,78536.0,175149500.0
9112,보르르분유포트,27330.0,4.0,0.00,95000.0,217185833.0


In [8]:
pd.options.display.max_rows = 100  

<font color="Maroon" size="5"><b>review / mall 등급 추출</b></font>

In [7]:
keywords = new_data["relative_keyword"]
keywords

0               대파크림치즈
1               열반사단열재
2           맥세이프실리콘케이스
3             에어프라이어용기
4                  커트보
             ...      
9109          닥터로운셀라큐어
9110              셀라큐어
9111    셀라큐어문제성발톱셀라솔루션
9112           보르르분유포트
9113              콩즈쥬스
Name: relative_keyword, Length: 9114, dtype: object

In [89]:
# tmp = pd.DataFrame(columns=["relative_keyword", 
#                             "grade_norm","grade_power","grade_bigpower","grade_premium",
#                            "review_10","review_50","review_100","review_1000",
#                            "total_review","avg_review"])
# tmp.to_csv("conditionedData/data/5_review_mallGrade/5_review_mallGrade_only_requests.csv",index=False)

In [10]:
cookies = {
    'NNB': '6TTK2RJU3JPWG',
    'autocomplete': 'use',
    '_ga': 'GA1.2.1198469368.1669035888',
    'NV_WETR_LOCATION_RGN_M': '"MDUxMTAxMTE="',
    'ASID': '6f5bbfa400000184fa368e5f00000067',
    'SHP_BUCKET_ID': '4',
    'NV_WETR_LAST_ACCESS_RGN_M': '"MDUxMTAxMTE="',
    'nx_ssl': '2',
    'nid_inf': '813506172',
    'NID_JKL': '7GO4JgsGQ6jSmPyQ+d0zZPEWBjZqV6VoPgsqFdP360s=',
    'NaverSuggestUse': 'unuse%26use',
    'NID_AUT': 'D2QOxk1i/qhN7mlsiYlMrQnnUV8Gl+BXV86T78q51HMwExxzB7wcbgptaPWfOS0c',
    'page_uid': 'iT2o/dqVOswssl/F5edssssstgG-438813',
    'NID_SES': 'AAABkxMPxjvBbsfunmw2moLJhwha50u3R6Eh8ZCx/2XDs3x7QGddf9U3CXn9gVSHzywkq0MispcT8m+dIy1l1yS0FIyGbYChnECtRPREzWdm49siY7QJT1jkkpdlFk6C16h9uf/rT/FB4fYE7R6ojElj2hMNIx0hZNJ1tI4jen6N20lm/UpZxFja6r75MpYe5OMSCTwKKh6yVjhl4YQCerCI/f8FCPuHZWJ+gBHohjLMgrbAe2BPEqOZn+gijX9P8EVGQNxlwMljKN75TowTPjpLqsWtS+wEHlQJiKPpKhSBpomRbQsSIqKUUdOBYDWRhrFai/A8+XWZDbMIa+QAlOELxnwqv20Qg7i3YG9qnfHebIg8mv9Fehy/dAWeg2Dyawq/CyhxzKkzw0kLiYbsqpRED6JfXY9mjyzLY0EmLf/urmflSfaHiyKrZ2P9DOSnaDnggiO7oPvP9f6AosTRuOxmTJCotPW6EPqOU436zFtG+x+jRjLgYJUDw76jfKY3y/XSnHMLOBmEc43jNMUccJMjbVEJEliEHhk1Mnt2nSeK3OBO',
    'spage_uid': 'iT2o%2FdqVOswssl%2FF5edssssstgG-438813',
    'sus_val': 'Lsbg/IOvghlD+lKI6nsg43tw',
    'ncpa': '3843648|lpnztn40|b78b4cf3e8deff6c21cfd6049a64dbba490b59a3|s_52e957ee25cb|a4ed494504da62925c00434d78bdc01c3c5768a4:494450|lpnzu3bc|33955d23247452d9f39b7befe57b1669f78a6422|s_223323393867677778|3739649221291330a79e03aba028f76e9b0939de:24|lpnzuyy8|eb0ed5873c0c6ebf2a6c5e362436a1ebf9eff7ce|s_419afe53a6bb|660b387db242b413e3d4d88cb426c264aa22cfb9:6506854|lpo0e5jc|f21ed58f1c6723c8cc4d00e3daa6a6422ffd0e32|s_202c49de12296|2945c3a090c4ef2300d5fd58af579649a7525b62:95694|lpo0g8ds|c5ac73032044e7c458f77e7f92f4cd0b391f5b71|95694|d5513b891cf3d61c63fc90b527f73eb7a2b1c655:367128|lpo0gbgw|c446229bc943cee174914f4c257713d2f205b404|s_172552030016403205|0ac152849d508c0f673b0dec0db0a8a25afb2d4b:1142874|lppah9xk|8c0bfc5cedba068684ebffb0372768a683b043e6|s_318df10bd1f2f|ddccaf77e58e8ff795e6da600b9730041ce0658a:1108389|lppbaieg|1a3ee417de104971a531073f1f19178ffd6a597e|s_2c6ae6bfc1750|78a17ad54d224316e33bd605f088b6eae27a49c0:8283684|lppgh5jc|da7cafc31641a6efb95608351d1829e697fd88f1|s_14ee9dbd76e3e|986bc50b91619ceced2d29619a58f16ee8584d8f:6|lpqs90ns|192ba3c47b3ad2ef4e536951fecef77a6e9dd7ec|s_265212fbc481|84b9a4af11a946f64d5d333ef74424f4381980c8',
}

headers = {
    'authority': 'search.shopping.naver.com',
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    # 'cookie': 'NNB=6TTK2RJU3JPWG; autocomplete=use; _ga=GA1.2.1198469368.1669035888; NV_WETR_LOCATION_RGN_M="MDUxMTAxMTE="; ASID=6f5bbfa400000184fa368e5f00000067; SHP_BUCKET_ID=4; NV_WETR_LAST_ACCESS_RGN_M="MDUxMTAxMTE="; nx_ssl=2; nid_inf=813506172; NID_JKL=7GO4JgsGQ6jSmPyQ+d0zZPEWBjZqV6VoPgsqFdP360s=; NaverSuggestUse=unuse%26use; NID_AUT=D2QOxk1i/qhN7mlsiYlMrQnnUV8Gl+BXV86T78q51HMwExxzB7wcbgptaPWfOS0c; page_uid=iT2o/dqVOswssl/F5edssssstgG-438813; NID_SES=AAABkxMPxjvBbsfunmw2moLJhwha50u3R6Eh8ZCx/2XDs3x7QGddf9U3CXn9gVSHzywkq0MispcT8m+dIy1l1yS0FIyGbYChnECtRPREzWdm49siY7QJT1jkkpdlFk6C16h9uf/rT/FB4fYE7R6ojElj2hMNIx0hZNJ1tI4jen6N20lm/UpZxFja6r75MpYe5OMSCTwKKh6yVjhl4YQCerCI/f8FCPuHZWJ+gBHohjLMgrbAe2BPEqOZn+gijX9P8EVGQNxlwMljKN75TowTPjpLqsWtS+wEHlQJiKPpKhSBpomRbQsSIqKUUdOBYDWRhrFai/A8+XWZDbMIa+QAlOELxnwqv20Qg7i3YG9qnfHebIg8mv9Fehy/dAWeg2Dyawq/CyhxzKkzw0kLiYbsqpRED6JfXY9mjyzLY0EmLf/urmflSfaHiyKrZ2P9DOSnaDnggiO7oPvP9f6AosTRuOxmTJCotPW6EPqOU436zFtG+x+jRjLgYJUDw76jfKY3y/XSnHMLOBmEc43jNMUccJMjbVEJEliEHhk1Mnt2nSeK3OBO; spage_uid=iT2o%2FdqVOswssl%2FF5edssssstgG-438813; sus_val=Lsbg/IOvghlD+lKI6nsg43tw; ncpa=3843648|lpnztn40|b78b4cf3e8deff6c21cfd6049a64dbba490b59a3|s_52e957ee25cb|a4ed494504da62925c00434d78bdc01c3c5768a4:494450|lpnzu3bc|33955d23247452d9f39b7befe57b1669f78a6422|s_223323393867677778|3739649221291330a79e03aba028f76e9b0939de:24|lpnzuyy8|eb0ed5873c0c6ebf2a6c5e362436a1ebf9eff7ce|s_419afe53a6bb|660b387db242b413e3d4d88cb426c264aa22cfb9:6506854|lpo0e5jc|f21ed58f1c6723c8cc4d00e3daa6a6422ffd0e32|s_202c49de12296|2945c3a090c4ef2300d5fd58af579649a7525b62:95694|lpo0g8ds|c5ac73032044e7c458f77e7f92f4cd0b391f5b71|95694|d5513b891cf3d61c63fc90b527f73eb7a2b1c655:367128|lpo0gbgw|c446229bc943cee174914f4c257713d2f205b404|s_172552030016403205|0ac152849d508c0f673b0dec0db0a8a25afb2d4b:1142874|lppah9xk|8c0bfc5cedba068684ebffb0372768a683b043e6|s_318df10bd1f2f|ddccaf77e58e8ff795e6da600b9730041ce0658a:1108389|lppbaieg|1a3ee417de104971a531073f1f19178ffd6a597e|s_2c6ae6bfc1750|78a17ad54d224316e33bd605f088b6eae27a49c0:8283684|lppgh5jc|da7cafc31641a6efb95608351d1829e697fd88f1|s_14ee9dbd76e3e|986bc50b91619ceced2d29619a58f16ee8584d8f:6|lpqs90ns|192ba3c47b3ad2ef4e536951fecef77a6e9dd7ec|s_265212fbc481|84b9a4af11a946f64d5d333ef74424f4381980c8',
    'logic': 'PART',
    'referer': 'https://search.shopping.naver.com/search/all?adQuery=%EB%B0%94%EB%94%94%ED%95%84%EB%A1%9C%EC%9A%B0&frm=NVSHATC&origQuery=%EB%B0%94%EB%94%94%ED%95%84%EB%A1%9C%EC%9A%B0&pagingIndex=1&pagingSize=40&productSet=total&query=%EB%B0%94%EB%94%94%ED%95%84%EB%A1%9C%EC%9A%B0&sort=rel&timestamp=&viewType=list',
    'sbth': 'be47be4da24bb869d213e377404c25cb6fdbd96cf65cc2343ae8c2a762ae17d1d81542acdaee8f278fc986fcbe3ffd2f',
    'sec-ch-ua': '"Google Chrome";v="119", "Chromium";v="119", "Not?A_Brand";v="24"',
    'sec-ch-ua-arch': '"x86"',
    'sec-ch-ua-bitness': '"64"',
    'sec-ch-ua-full-version-list': '"Google Chrome";v="119.0.6045.200", "Chromium";v="119.0.6045.200", "Not?A_Brand";v="24.0.0.0"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-model': '""',
    'sec-ch-ua-platform': '"Windows"',
    'sec-ch-ua-platform-version': '"15.0.0"',
    'sec-ch-ua-wow64': '?0',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
}

In [91]:
# df_review_mallGrade = pd.read_csv("conditionedData/data/5_review_mallGrade/5_review_mallGrade_only_requests.csv")
# df_review_mallGrade

,relative_keyword,grade_norm,grade_power,grade_bigpower,grade_premium,review_10,review_50,review_100,review_1000,total_review,avg_review


In [11]:
df_review_mallGrade = pd.read_csv("conditionedData/data/5_review_mallGrade/5_review_mallGrade_only_requests.csv")

num = 1845

for keyword in keywords[num:]:
    
    params = {
        'adQuery': f'{keyword}',
        'eq': '',
        'frm': 'NVSHATC',
        'iq': '',
        'origQuery': f'{keyword}',
        'pagingIndex': '1',
        'pagingSize': '40',
        'productSet': 'total',
        'query': f'{keyword}',
        'sort': 'rel',
        'viewType': 'list',
        'xq': '',
    }

    response = requests.get('https://search.shopping.naver.com/api/search/all', params=params, cookies=cookies, headers=headers)
    itemlists = json.loads(response.text)
    itemlists

    """
    리뷰 ( 10-50-100-1000 ) 
    빅파워 개수 (일반 - 파워 - 빅파워 - 플래티넘/프리미엄 )
    *광고는 개수에 포함하지 않음*
    """

    grade_norm = 0
    grade_power = 0
    grade_bigpower = 0
    grade_premium = 0

    cnt = 0
    total_review = 0

    review_10 = 0
    review_50 = 0
    review_100 = 0
    review_1000 = 0


    for idx,itemlist  in enumerate(itemlists["shoppingResult"]["products"]) :

        reviewCount = itemlist["reviewCount"]

        # review 개수 나누기  + 평균
        if (reviewCount > 1000):
            review_1000 += 1
        elif (reviewCount > 100) :  
            review_100 += 1
        elif (reviewCount > 50) :  
            review_50 += 1
        else : review_10 += 1

        total_review += reviewCount
        cnt += 1  # 평균 구하기 위함


        try: 
            mallGrade = itemlist["mallInfoCache"]["mallGrade"]

            # mall 등급 개수 파익
            if(mallGrade=="M44001" or mallGrade=="M44002") :
                grade_premium += 1
            elif (mallGrade=="M44003"): 
                grade_bigpower += 1
            elif (mallGrade=="M44004"): 
                grade_power += 1
            else :
                grade_norm += 1

        except:
            pass
        
    print(f"{num}.{keyword}")
    print(f" 일반:{grade_norm} | 파워:{grade_power} | 빅파워:{grade_bigpower} | 플래/프리미엄:{grade_premium}")
    print(f"50개이하:{review_10} | 50개 이상:{review_50} | 100개 이상 : {review_100} | 1000개 이상:{review_1000}")
    print(f"전체 리뷰:{total_review} | 평균 리뷰:{int(total_review/cnt)}" )
    print("----------------------------------------------------------------------")
        
    # DataFrame 만들기
    tmp = pd.DataFrame({
            "relative_keyword":keyword, 
            "grade_norm":[grade_norm],"grade_power":grade_power,"grade_bigpower":[grade_bigpower],"grade_premium":[grade_premium],
            "review_10":[review_10],"review_50":[review_50],"review_100":[review_100],"review_1000":[review_1000],
            "total_review":[total_review],"avg_review":[int(total_review/cnt)]})    
    
    df_review_mallGrade = pd.concat([df_review_mallGrade,tmp],axis=0)
        
    num += 1 # keyword index
    df_review_mallGrade.to_csv("conditionedData/data/5_review_mallGrade/5_review_mallGrade_only_requests.csv",index=False)
    
            


1845.픽디자인스트랩
 일반:9 | 파워:1 | 빅파워:21 | 플래/프리미엄:5
50개이하:29 | 50개 이상:2 | 100개 이상 : 8 | 1000개 이상:1
전체 리뷰:4660 | 평균 리뷰:116
----------------------------------------------------------------------
1846.한샘빨래바구니
 일반:0 | 파워:7 | 빅파워:10 | 플래/프리미엄:10
50개이하:31 | 50개 이상:3 | 100개 이상 : 6 | 1000개 이상:0
전체 리뷰:2528 | 평균 리뷰:63
----------------------------------------------------------------------
1847.시그니아보청기
 일반:5 | 파워:1 | 빅파워:1 | 플래/프리미엄:4
50개이하:31 | 50개 이상:4 | 100개 이상 : 4 | 1000개 이상:1
전체 리뷰:13007 | 평균 리뷰:325
----------------------------------------------------------------------
1848.태닝로션
 일반:12 | 파워:5 | 빅파워:14 | 플래/프리미엄:2
50개이하:16 | 50개 이상:5 | 100개 이상 : 15 | 1000개 이상:4
전체 리뷰:12235 | 평균 리뷰:305
----------------------------------------------------------------------
1849.욕창방석
 일반:4 | 파워:5 | 빅파워:3 | 플래/프리미엄:4
50개이하:29 | 50개 이상:3 | 100개 이상 : 3 | 1000개 이상:5
전체 리뷰:9757 | 평균 리뷰:243
----------------------------------------------------------------------
1850.냉동망고
 일반:2 | 파워:3 | 빅파워:24 | 플래/프리미엄:11
50개이하:3 | 50개 이상:3 

1889.쌍커플테이프
 일반:8 | 파워:11 | 빅파워:10 | 플래/프리미엄:10
50개이하:6 | 50개 이상:2 | 100개 이상 : 17 | 1000개 이상:15
전체 리뷰:101529 | 평균 리뷰:2538
----------------------------------------------------------------------
1890.쌍커풀테이프
 일반:8 | 파워:12 | 빅파워:10 | 플래/프리미엄:9
50개이하:6 | 50개 이상:3 | 100개 이상 : 16 | 1000개 이상:15
전체 리뷰:100969 | 평균 리뷰:2524
----------------------------------------------------------------------
1891.쌍꺼풀테이프
 일반:8 | 파워:12 | 빅파워:10 | 플래/프리미엄:9
50개이하:6 | 50개 이상:3 | 100개 이상 : 16 | 1000개 이상:15
전체 리뷰:100969 | 평균 리뷰:2524
----------------------------------------------------------------------
1892.블루보틀텀블러
 일반:9 | 파워:9 | 빅파워:9 | 플래/프리미엄:6
50개이하:23 | 50개 이상:4 | 100개 이상 : 11 | 1000개 이상:2
전체 리뷰:7571 | 평균 리뷰:189
----------------------------------------------------------------------
1893.공간분리커튼
 일반:7 | 파워:7 | 빅파워:9 | 플래/프리미엄:1
50개이하:11 | 50개 이상:4 | 100개 이상 : 18 | 1000개 이상:7
전체 리뷰:22173 | 평균 리뷰:554
----------------------------------------------------------------------
1894.전화신점
 일반:22 | 파워:6 | 빅파워:12 | 플래/프리미엄:0
50

1933.핸드타올디스펜서
 일반:0 | 파워:2 | 빅파워:11 | 플래/프리미엄:5
50개이하:10 | 50개 이상:4 | 100개 이상 : 19 | 1000개 이상:7
전체 리뷰:27057 | 평균 리뷰:676
----------------------------------------------------------------------
1934.꼬들빼기김치
 일반:7 | 파워:9 | 빅파워:15 | 플래/프리미엄:7
50개이하:21 | 50개 이상:4 | 100개 이상 : 12 | 1000개 이상:3
전체 리뷰:15810 | 평균 리뷰:395
----------------------------------------------------------------------
1935.퓨전한복
 일반:2 | 파워:5 | 빅파워:9 | 플래/프리미엄:0
50개이하:35 | 50개 이상:4 | 100개 이상 : 1 | 1000개 이상:0
전체 리뷰:1153 | 평균 리뷰:28
----------------------------------------------------------------------
1936.주니어패딩조끼
 일반:3 | 파워:5 | 빅파워:13 | 플래/프리미엄:2
50개이하:35 | 50개 이상:3 | 100개 이상 : 2 | 1000개 이상:0
전체 리뷰:1444 | 평균 리뷰:36
----------------------------------------------------------------------
1937.스텐웍
 일반:6 | 파워:3 | 빅파워:14 | 플래/프리미엄:10
50개이하:25 | 50개 이상:3 | 100개 이상 : 11 | 1000개 이상:1
전체 리뷰:6780 | 평균 리뷰:169
----------------------------------------------------------------------
1938.MTS롤러
 일반:13 | 파워:7 | 빅파워:10 | 플래/프리미엄:2
50개이하:11 | 50개 이상:

1977.아동패딩조끼
 일반:2 | 파워:3 | 빅파워:14 | 플래/프리미엄:2
50개이하:31 | 50개 이상:3 | 100개 이상 : 5 | 1000개 이상:1
전체 리뷰:4714 | 평균 리뷰:117
----------------------------------------------------------------------
1978.쿠쿠밥솥고무패킹
 일반:1 | 파워:0 | 빅파워:13 | 플래/프리미엄:26
50개이하:7 | 50개 이상:3 | 100개 이상 : 25 | 1000개 이상:5
전체 리뷰:20813 | 평균 리뷰:520
----------------------------------------------------------------------
1979.로얄캐닌가스트로인테스티널로우펫
 일반:5 | 파워:5 | 빅파워:12 | 플래/프리미엄:16
50개이하:24 | 50개 이상:5 | 100개 이상 : 8 | 1000개 이상:3
전체 리뷰:18215 | 평균 리뷰:455
----------------------------------------------------------------------
1980.판게아인섹트
 일반:9 | 파워:5 | 빅파워:13 | 플래/프리미엄:0
50개이하:25 | 50개 이상:5 | 100개 이상 : 10 | 1000개 이상:0
전체 리뷰:3418 | 평균 리뷰:85
----------------------------------------------------------------------
1981.보웰담요
 일반:0 | 파워:5 | 빅파워:3 | 플래/프리미엄:12
50개이하:31 | 50개 이상:4 | 100개 이상 : 5 | 1000개 이상:0
전체 리뷰:2083 | 평균 리뷰:52
----------------------------------------------------------------------
1982.젖병보관함
 일반:5 | 파워:6 | 빅파워:8 | 플래/프리미엄:3
50개이하:23

2021.피스복
 일반:1 | 파워:2 | 빅파워:3 | 플래/프리미엄:0
50개이하:28 | 50개 이상:6 | 100개 이상 : 5 | 1000개 이상:1
전체 리뷰:4640 | 평균 리뷰:116
----------------------------------------------------------------------
2022.세탁실수납장
 일반:6 | 파워:10 | 빅파워:16 | 플래/프리미엄:2
50개이하:19 | 50개 이상:1 | 100개 이상 : 13 | 1000개 이상:7
전체 리뷰:20872 | 평균 리뷰:521
----------------------------------------------------------------------
2023.김국산마스크
 일반:0 | 파워:6 | 빅파워:12 | 플래/프리미엄:20
50개이하:22 | 50개 이상:1 | 100개 이상 : 11 | 1000개 이상:6
전체 리뷰:149638 | 평균 리뷰:3740
----------------------------------------------------------------------
2024.목발
 일반:2 | 파워:2 | 빅파워:1 | 플래/프리미엄:2
50개이하:30 | 50개 이상:2 | 100개 이상 : 6 | 1000개 이상:2
전체 리뷰:5081 | 평균 리뷰:127
----------------------------------------------------------------------
2025.우설
 일반:8 | 파워:8 | 빅파워:20 | 플래/프리미엄:4
50개이하:27 | 50개 이상:6 | 100개 이상 : 7 | 1000개 이상:0
전체 리뷰:2126 | 평균 리뷰:53
----------------------------------------------------------------------
2026.락토프리우유
 일반:0 | 파워:7 | 빅파워:12 | 플래/프리미엄:11
50개이하:13 | 50개 이상:1 | 10

...

8929.무화과제철
 일반:7 | 파워:3 | 빅파워:2 | 플래/프리미엄:28
50개이하:38 | 50개 이상:1 | 100개 이상 : 0 | 1000개 이상:1
전체 리뷰:2788 | 평균 리뷰:69
----------------------------------------------------------------------
8930.김치10KG
 일반:1 | 파워:1 | 빅파워:14 | 플래/프리미엄:3
50개이하:1 | 50개 이상:0 | 100개 이상 : 8 | 1000개 이상:31
전체 리뷰:316547 | 평균 리뷰:7913
----------------------------------------------------------------------
8931.아던샵속눈썹펌
 일반:2 | 파워:0 | 빅파워:7 | 플래/프리미엄:29
50개이하:23 | 50개 이상:2 | 100개 이상 : 10 | 1000개 이상:5
전체 리뷰:174371 | 평균 리뷰:4359
----------------------------------------------------------------------
8932.메디토이
 일반:14 | 파워:1 | 빅파워:5 | 플래/프리미엄:9
50개이하:22 | 50개 이상:1 | 100개 이상 : 5 | 1000개 이상:6
전체 리뷰:115859 | 평균 리뷰:3407
----------------------------------------------------------------------
8933.한스킨클렌징오일
 일반:12 | 파워:5 | 빅파워:3 | 플래/프리미엄:16
50개이하:31 | 50개 이상:1 | 100개 이상 : 4 | 1000개 이상:4
전체 리뷰:50071 | 평균 리뷰:1251
----------------------------------------------------------------------
8934.강아지알러지사료
 일반:0 | 파워:1 | 빅파워:10 | 플래/프리미엄:7
50개이하

8973.부사사과
 일반:4 | 파워:11 | 빅파워:23 | 플래/프리미엄:2
50개이하:1 | 50개 이상:1 | 100개 이상 : 13 | 1000개 이상:25
전체 리뷰:339760 | 평균 리뷰:8494
----------------------------------------------------------------------
8974.베베데코에어컨커버
 일반:1 | 파워:0 | 빅파워:7 | 플래/프리미엄:7
50개이하:24 | 50개 이상:4 | 100개 이상 : 7 | 1000개 이상:5
전체 리뷰:86455 | 평균 리뷰:2161
----------------------------------------------------------------------
8975.보다나봉고데기40MM
 일반:7 | 파워:0 | 빅파워:2 | 플래/프리미엄:26
50개이하:27 | 50개 이상:4 | 100개 이상 : 5 | 1000개 이상:4
전체 리뷰:12023 | 평균 리뷰:300
----------------------------------------------------------------------
8976.수직목견인기
 일반:2 | 파워:1 | 빅파워:3 | 플래/프리미엄:19
50개이하:38 | 50개 이상:0 | 100개 이상 : 1 | 1000개 이상:1
전체 리뷰:5302 | 평균 리뷰:132
----------------------------------------------------------------------
8977.광주서울곱창
 일반:0 | 파워:0 | 빅파워:2 | 플래/프리미엄:0
50개이하:0 | 50개 이상:0 | 100개 이상 : 0 | 1000개 이상:3
전체 리뷰:7215 | 평균 리뷰:2405
----------------------------------------------------------------------
8978.아임반허리쿠션
 일반:0 | 파워:1 | 빅파워:2 | 플래/프리미엄:7
50개이하:8

9017.남도의식탁
 일반:3 | 파워:0 | 빅파워:0 | 플래/프리미엄:37
50개이하:37 | 50개 이상:0 | 100개 이상 : 2 | 1000개 이상:1
전체 리뷰:11041 | 평균 리뷰:276
----------------------------------------------------------------------
9018.키토우보풀제거기
 일반:4 | 파워:0 | 빅파워:4 | 플래/프리미엄:32
50개이하:37 | 50개 이상:1 | 100개 이상 : 1 | 1000개 이상:1
전체 리뷰:2816 | 평균 리뷰:70
----------------------------------------------------------------------
9019.퓨어닷소독티슈
 일반:1 | 파워:1 | 빅파워:0 | 플래/프리미엄:37
50개이하:35 | 50개 이상:1 | 100개 이상 : 3 | 1000개 이상:1
전체 리뷰:2336 | 평균 리뷰:58
----------------------------------------------------------------------
9020.달바선크림
 일반:3 | 파워:4 | 빅파워:10 | 플래/프리미엄:18
50개이하:24 | 50개 이상:4 | 100개 이상 : 7 | 1000개 이상:5
전체 리뷰:236688 | 평균 리뷰:5917
----------------------------------------------------------------------
9021.말랑하니방수패드
 일반:0 | 파워:1 | 빅파워:0 | 플래/프리미엄:15
50개이하:14 | 50개 이상:2 | 100개 이상 : 2 | 1000개 이상:1
전체 리뷰:25777 | 평균 리뷰:1356
----------------------------------------------------------------------
9022.아기수딩젤
 일반:0 | 파워:0 | 빅파워:1 | 플래/프리미엄:6
50개이하:0 | 50개

9061.인포벨요술짤순이
 일반:4 | 파워:1 | 빅파워:3 | 플래/프리미엄:15
50개이하:20 | 50개 이상:0 | 100개 이상 : 2 | 1000개 이상:2
전체 리뷰:2752 | 평균 리뷰:114
----------------------------------------------------------------------
9062.과일꾼
 일반:0 | 파워:0 | 빅파워:7 | 플래/프리미엄:31
50개이하:14 | 50개 이상:4 | 100개 이상 : 11 | 1000개 이상:11
전체 리뷰:151699 | 평균 리뷰:3792
----------------------------------------------------------------------
9063.푸른회식당
 일반:3 | 파워:0 | 빅파워:1 | 플래/프리미엄:14
50개이하:18 | 50개 이상:0 | 100개 이상 : 0 | 1000개 이상:1
전체 리뷰:21946 | 평균 리뷰:1155
----------------------------------------------------------------------
9064.써유
 일반:8 | 파워:1 | 빅파워:19 | 플래/프리미엄:11
50개이하:26 | 50개 이상:2 | 100개 이상 : 3 | 1000개 이상:9
전체 리뷰:115897 | 평균 리뷰:2897
----------------------------------------------------------------------
9065.본베베침대가드
 일반:0 | 파워:1 | 빅파워:3 | 플래/프리미엄:23
50개이하:25 | 50개 이상:0 | 100개 이상 : 3 | 1000개 이상:2
전체 리뷰:84701 | 평균 리뷰:2823
----------------------------------------------------------------------
9066.착한미식가
 일반:2 | 파워:0 | 빅파워:2 | 플래/프리미엄:6
50개이하:8 | 50개

9105.폴레드샤워핸들
 일반:0 | 파워:0 | 빅파워:2 | 플래/프리미엄:29
50개이하:31 | 50개 이상:4 | 100개 이상 : 2 | 1000개 이상:3
전체 리뷰:21600 | 평균 리뷰:540
----------------------------------------------------------------------
9106.써유모래
 일반:11 | 파워:4 | 빅파워:10 | 플래/프리미엄:13
50개이하:29 | 50개 이상:1 | 100개 이상 : 0 | 1000개 이상:10
전체 리뷰:112295 | 평균 리뷰:2807
----------------------------------------------------------------------
9107.노시부프로
 일반:3 | 파워:2 | 빅파워:4 | 플래/프리미엄:25
50개이하:24 | 50개 이상:0 | 100개 이상 : 5 | 1000개 이상:11
전체 리뷰:135995 | 평균 리뷰:3399
----------------------------------------------------------------------
9108.힘가네생강청
 일반:5 | 파워:0 | 빅파워:3 | 플래/프리미엄:19
50개이하:23 | 50개 이상:1 | 100개 이상 : 2 | 1000개 이상:1
전체 리뷰:35109 | 평균 리뷰:1300
----------------------------------------------------------------------
9109.닥터로운셀라큐어
 일반:0 | 파워:0 | 빅파워:1 | 플래/프리미엄:13
50개이하:13 | 50개 이상:1 | 100개 이상 : 0 | 1000개 이상:1
전체 리뷰:8317 | 평균 리뷰:554
----------------------------------------------------------------------
9110.셀라큐어
 일반:1 | 파워:1 | 빅파워:6 | 플래/프리미엄:29
50개이하:37

<font color="Maroon" size="5"><b>review_mallGrade 합치기</b></font>

In [9]:
df_review_mallGrade = pd.read_csv("conditionedData/data/5_review_mallGrade/5_review_mallGrade_only_requests.csv")
df_review_mallGrade

,relative_keyword,grade_norm,grade_power,grade_bigpower,grade_premium,review_10,review_50,review_100,review_1000,total_review,avg_review
0,대파크림치즈,9,3,2,5,36,2,2,0,983,24
1,열반사단열재,2,5,27,6,15,2,18,5,22774,569
2,맥세이프실리콘케이스,2,0,12,10,13,5,19,3,13854,346
3,에어프라이어용기,2,5,5,3,16,2,15,7,26171,654
4,커트보,9,7,18,6,11,4,19,6,20997,524
...,...,...,...,...,...,...,...,...,...,...,...
9109,닥터로운셀라큐어,0,0,1,13,13,1,0,1,8317,554
9110,셀라큐어,1,1,6,29,37,1,1,1,8804,220
9111,셀라큐어문제성발톱셀라솔루션,0,1,6,27,32,1,1,1,8827,252
9112,보르르분유포트,0,0,0,3,1,0,1,2,115990,28997


In [16]:
data = pd.merge(new_data,df_review_mallGrade, on="relative_keyword")
data.to_csv("conditionedData/data/5_review_mallGrade/5_review_mallGrade_requests.csv",index=False)   

In [17]:
pd.read_csv("conditionedData/data/5_review_mallGrade/5_review_mallGrade_requests.csv")

,relative_keyword,total_search,total_store_count,competitive_rate,avg_price,avg_sale,grade_norm,grade_power,grade_bigpower,grade_premium,review_10,review_50,review_100,review_1000,total_review,avg_review
0,대파크림치즈,10330.0,1375.0,0.13,17500.0,528041.0,9,3,2,5,36,2,2,0,983,24
1,열반사단열재,5750.0,8235.0,1.43,48356.0,528275.0,2,5,27,6,15,2,18,5,22774,569
2,맥세이프실리콘케이스,1470.0,92439.0,62.88,33779.0,528374.0,2,0,12,10,13,5,19,3,13854,346
3,에어프라이어용기,2730.0,41153.0,15.07,12648.0,528417.0,2,5,5,3,16,2,15,7,26171,654
4,커트보,2390.0,58518.0,24.48,11314.0,528460.0,9,7,18,6,11,4,19,6,20997,524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9109,닥터로운셀라큐어,1590.0,18.0,0.01,34450.0,175149500.0,0,0,1,13,13,1,0,1,8317,554
9110,셀라큐어,10750.0,107.0,0.01,47172.0,175149500.0,1,1,6,29,37,1,1,1,8804,220
9111,셀라큐어문제성발톱셀라솔루션,2930.0,43.0,0.01,78536.0,175149500.0,0,1,6,27,32,1,1,1,8827,252
9112,보르르분유포트,27330.0,4.0,0.00,95000.0,217185833.0,0,0,0,3,1,0,1,2,115990,28997


<font color="Maroon" size="5"><b>단일 셀 TEST</b></font>

In [112]:
keyword  = "홍쌍리매실장아찌"
params = {
        'adQuery': f'{keyword}',
        'eq': '',
        'frm': 'NVSHATC',
        'iq': '',
        'origQuery': f'{keyword}',
        'pagingIndex': '1',
         'pagingSize': '40',
        'productSet': 'total',
        'query': f'{keyword}',
        'sort': 'rel',
        'viewType': 'list',
        'xq': '',
    }

response = requests.get('https://search.shopping.naver.com/api/search/all', params=params, cookies=cookies, headers=headers)
itemlists = json.loads(response.text)
itemlists

"""
리뷰 ( 10-50-100-1000 ) 
빅파워 개수 (일반 - 파워 - 빅파워 - 플래티넘/프리미엄 )
*광고는 개수에 포함하지 않음*
"""

grade_norm = 0
grade_power = 0
grade_bigpower = 0
grade_premium = 0

cnt = 0
total_review = 0

review_10 = 0
review_50 = 0
review_100 = 0
review_1000 = 0


for idx,itemlist  in enumerate(itemlists["shoppingResult"]["products"]) :

    reviewCount = itemlist["reviewCount"]

    # review 개수 나누기  + 평균
    if (reviewCount > 1000):
        review_1000 += 1
    elif (reviewCount > 100) :  
        review_100 += 1
    elif (reviewCount > 50) :  
        review_50 += 1
    else : review_10 += 1

    total_review += reviewCount
    cnt += 1  # 평균 구하기 위함
    

    try: 
        mallGrade = itemlist["mallInfoCache"]["mallGrade"]
        prin t(mallGrade)

        # mall 등급 개수 파익
        if(mallGrade=="M44001" or mallGrade=="M44002") :
            grade_premium += 1
        elif (mallGrade=="M44003"): 
            grade_bigpower += 1
        elif (mallGrade=="M44004"): 
            grade_power += 1
        else :
            grade_norm += 1

    except:
        pass

print(f"{num}.{keyword}")
print(f" 일반:{grade_norm} | 파워:{grade_power} | 빅파워:{grade_bigpower} | 플래/프리미엄:{grade_premium}")
print(f"50개이하:{review_10} | 50개 이상:{review_50} | 100개 이상 : {review_100} | 1000개 이상:{review_1000}")
print(f"전체 리뷰:{total_review} | 평균 리뷰:{int(total_review/cnt)}" )
print("-----------------------------------------------------")

M44003
M44003
M44003
M44003
M44005
M44005
M44004
M44001
M44005
M44004
M44003
M44001
508.홍쌍리매실장아찌
 일반:3 | 파워:2 | 빅파워:5 | 플래/프리미엄:2
50개이하:33 | 50개 이상:1 | 100개 이상 : 5 | 1000개 이상:1
전체 리뷰:2822 | 평균 리뷰:70
-----------------------------------------------------


In [106]:
mallGrade

'M44003'

In [29]:
import json
import requests

cookies = {
    'NNB': '6TTK2RJU3JPWG',
    'autocomplete': 'use',
    '_ga': 'GA1.2.1198469368.1669035888',
    'NV_WETR_LOCATION_RGN_M': '"MDUxMTAxMTE="',
    'ASID': '6f5bbfa400000184fa368e5f00000067',
    'SHP_BUCKET_ID': '4',
    'NV_WETR_LAST_ACCESS_RGN_M': '"MDUxMTAxMTE="',
    'nx_ssl': '2',
    'nid_inf': '813506172',
    'NID_AUT': '6kiAbtWvFZK3y90sHs9cfKS12yY1zkUBJdBmlEKIM2kq2fWAmKgi9OnXOnz9nUyL',
    'NID_JKL': '7GO4JgsGQ6jSmPyQ+d0zZPEWBjZqV6VoPgsqFdP360s=',
    'page_uid': 'iTrAvdqVOZCssRoVy7Vssssstcl-188131',
    'spage_uid': 'iTrAvdqVOZCssRoVy7Vssssstcl-188131',
    'NID_SES': 'AAABhRMGcG38dd+AVbQV0majyYVy//mj0pMGIdwEgF50kuXMG7yzS7NgEWATTyWEDz+h3igfc8lmNMdqxPT3hNZiIXQyGAPpPlJaomIY1eyU8eMG1nMOIQDWyQqZUkNapJKyXwt1ZsBZYkQlba3no2oFokuyLoL/TWn8p2i7tGNYdeKCT6J/RW0prfsCZuDRlIR+aXy0s48Wp6jB0j9xQM9Swzt2BxD2EBPvUvZJVAqyQKrjoPYsxhBo0mwd0C3L2oxnQUPG1bx2IlAs8dFEtoAz17Kx6YxF2b0XPVs8ffK8ZGgHs6192PTpVLk2IVtSKa5BFZN6Q7fgBePj/WZ0XCLp8s2BxGw2qdl3IXeezVA3f3Xl46B2cuZQPeEUKrZUS7tIFdbwiDR8LPrue/e/EK2ZZsQIzyayOuzeEXJYshF0ZD2UOGULTWv2NhkLVeLKqd1hq3oBPxX/wBkQjeKBqeJde8OjOG980cj+qRVCbCsmedoqrwP+yLta99Ksmt7res+DArlvf8PLzrHVb7ZYrCcFF0E=',
    'ncpa': '8906154|lpl7awf4|42d1baece663e0d213ba404621cf481b7242f35f|s_e26988f9dc99|6c2d64a4c1df76e2280336de7f219a7a355468a1:182410|lpl8m8w8|15c34379acb36244a25e6884b137d9e6e5db97d7|s_1eeb12ba8dfe6bdf|a5e20b94bc6a7baaa057cd3c57fea5e0848efe86:24|lpmnyhxk|fa6bbca73f1d69f3e74162fd22aa9cd128b177b2|s_419afe53a6bb|7cb0f74e460fe247033a56e43cca0af5be4fb347:95694|lpnwg4vs|6bc56e225781b65431a50241c5ef46e62183461b|95694|c9653a506f034c066b2b000523448796fc82c83c',
    'NaverSuggestUse': 'unuse%26use',
}

headers = {
    'authority': 'search.shopping.naver.com',
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'content-type': 'application/json',
    # 'cookie': 'NNB=6TTK2RJU3JPWG; autocomplete=use; _ga=GA1.2.1198469368.1669035888; NV_WETR_LOCATION_RGN_M="MDUxMTAxMTE="; ASID=6f5bbfa400000184fa368e5f00000067; SHP_BUCKET_ID=4; NV_WETR_LAST_ACCESS_RGN_M="MDUxMTAxMTE="; nx_ssl=2; nid_inf=813506172; NID_AUT=6kiAbtWvFZK3y90sHs9cfKS12yY1zkUBJdBmlEKIM2kq2fWAmKgi9OnXOnz9nUyL; NID_JKL=7GO4JgsGQ6jSmPyQ+d0zZPEWBjZqV6VoPgsqFdP360s=; page_uid=iTrAvdqVOZCssRoVy7Vssssstcl-188131; spage_uid=iTrAvdqVOZCssRoVy7Vssssstcl-188131; NID_SES=AAABhRMGcG38dd+AVbQV0majyYVy//mj0pMGIdwEgF50kuXMG7yzS7NgEWATTyWEDz+h3igfc8lmNMdqxPT3hNZiIXQyGAPpPlJaomIY1eyU8eMG1nMOIQDWyQqZUkNapJKyXwt1ZsBZYkQlba3no2oFokuyLoL/TWn8p2i7tGNYdeKCT6J/RW0prfsCZuDRlIR+aXy0s48Wp6jB0j9xQM9Swzt2BxD2EBPvUvZJVAqyQKrjoPYsxhBo0mwd0C3L2oxnQUPG1bx2IlAs8dFEtoAz17Kx6YxF2b0XPVs8ffK8ZGgHs6192PTpVLk2IVtSKa5BFZN6Q7fgBePj/WZ0XCLp8s2BxGw2qdl3IXeezVA3f3Xl46B2cuZQPeEUKrZUS7tIFdbwiDR8LPrue/e/EK2ZZsQIzyayOuzeEXJYshF0ZD2UOGULTWv2NhkLVeLKqd1hq3oBPxX/wBkQjeKBqeJde8OjOG980cj+qRVCbCsmedoqrwP+yLta99Ksmt7res+DArlvf8PLzrHVb7ZYrCcFF0E=; ncpa=8906154|lpl7awf4|42d1baece663e0d213ba404621cf481b7242f35f|s_e26988f9dc99|6c2d64a4c1df76e2280336de7f219a7a355468a1:182410|lpl8m8w8|15c34379acb36244a25e6884b137d9e6e5db97d7|s_1eeb12ba8dfe6bdf|a5e20b94bc6a7baaa057cd3c57fea5e0848efe86:24|lpmnyhxk|fa6bbca73f1d69f3e74162fd22aa9cd128b177b2|s_419afe53a6bb|7cb0f74e460fe247033a56e43cca0af5be4fb347:95694|lpnwg4vs|6bc56e225781b65431a50241c5ef46e62183461b|95694|c9653a506f034c066b2b000523448796fc82c83c; NaverSuggestUse=unuse%26use',
    'origin': 'https://search.shopping.naver.com',
    'referer': 'https://search.shopping.naver.com/search/all?query=%EC%95%A1%EC%A0%95%ED%95%84%EB%A6%84&cat_id=&frm=NVSHATC',
    'sbth': '77074ab4693c2007419f7fba27a0d1de44c43a4f3d9d2d265814254b5a74210643c3a3a92780474e1ddc1b64fdf8a0fd',
    'sec-ch-ua': '"Google Chrome";v="119", "Chromium";v="119", "Not?A_Brand";v="24"',
    'sec-ch-ua-arch': '"x86"',
    'sec-ch-ua-bitness': '"64"',
    'sec-ch-ua-full-version-list': '"Google Chrome";v="119.0.6045.200", "Chromium";v="119.0.6045.200", "Not?A_Brand";v="24.0.0.0"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-model': '""',
    'sec-ch-ua-platform': '"Windows"',
    'sec-ch-ua-platform-version': '"15.0.0"',
    'sec-ch-ua-wow64': '?0',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
}

json_data = {
    'params': [
        {
            'nvMid': '29904046618',
            'chpid': None,
            'isBook': False,
        },
        {
            'nvMid': '36844185618',
            'chpid': None,
            'isBook': False,
        },
        {
            'nvMid': '35708856618',
            'chpid': None,
            'isBook': False,
        },
        {
            'nvMid': '86822771096',
            'chpid': '9278270773',
            'isBook': False,
        },
        {
            'nvMid': '42649250618',
            'chpid': None,
            'isBook': False,
        },
        {
            'nvMid': '24564590522',
            'chpid': None,
            'isBook': False,
        },
        {
            'nvMid': '40307031618',
            'chpid': None,
            'isBook': False,
        },
        {
            'nvMid': '83780027994',
            'chpid':  None,
            'isBook': False,
        },
        {
            'nvMid': '84926693078',
            'chpid': None,
            'isBook': False,
        },
        {
            'nvMid': '84391743563',
            'chpid': None,
            'isBook': False,
        },
    ],
}

response = requests.post(
    'https://search.shopping.naver.com/api/product-zzim/products',
    cookies=cookies,
    headers=headers,
    json=json_data,
)

# Note: json_data will not be serialized by requests
# exactly as it was in the original request.
#data = '{"params":[{"nvMid":"29904046618","chpid":null,"isBook":false},{"nvMid":"36844185618","chpid":null,"isBook":false},{"nvMid":"35708856618","chpid":null,"isBook":false},{"nvMid":"86822771096","chpid":"9278270773","isBook":false},{"nvMid":"42649250618","chpid":null,"isBook":false},{"nvMid":"24564590522","chpid":null,"isBook":false},{"nvMid":"40307031618","chpid":null,"isBook":false},{"nvMid":"83780027994","chpid":"6235525505","isBook":false},{"nvMid":"84926693078","chpid":"7382192756","isBook":false},{"nvMid":"84391743563","chpid":"6847243241","isBook":false}]}'
#response = requests.post(
#    'https://search.shopping.naver.com/api/product-zzim/products',
#    cookies=cookies,
#    headers=headers,
#    data=data,
#)

In [30]:
print(response.text)

{"bSuccess":true,"zzim":{"86822771096":{"isZzim":false,"count":377},"83780027994":{"isZzim":false,"count":1041},"84391743563":{"isZzim":false,"count":162},"84926693078":{"isZzim":false,"count":22},"29904046618":{"isZzim":false,"count":8},"36844185618":{"isZzim":false,"count":199},"35708856618":{"isZzim":false,"count":102},"42649250618":{"isZzim":false,"count":43},"24564590522":{"isZzim":false,"count":13},"40307031618":{"isZzim":false,"count":43}}}


In [21]:
response

<Response [200]>

In [38]:
import requests
from bs4 import BeautifulSoup
import json

response = requests.get("https://search.shopping.naver.com/search/all?query=%EC%82%AC%EA%B3%BC&bt=1&frm=NVSCPRO")
html = response.text
soup = BeautifulSoup(html,"html.parser")
links = soup.select("#__NEXT_DATA__")
data = json.loads(links[0].text)
data

{'props': {'pageProps': {'ogTag': {'title': '사과 : 네이버 쇼핑',
    'image': 'https://ssl.pstatic.net/imgshopping/static/mo2014/icon/og_shopping_v2.png',
    'description': "'사과'의 네이버쇼핑 검색 결과입니다"},
   'bucketId': '',
   'userInfo': {'demoType': 'A00',
    'naverIdNo': '',
    'naverId': '',
    'nickName': '',
    'isAdultAge': False,
    'isRealAdult': False,
    'naverMembershipEndDate': '',
    'isValidMembership': False,
    'isFemale': False,
    'isMale': False,
    'koreanAge': None,
    'age': None,
    'nnb': ''},
   'cartProductCount': 0,
   'dateStatus': {'date': '2023-12-03T12:03:12.218Z',
    'isHoliday': False,
    'isWeekend': True,
    'dayOfWeek': '일',
    'dayOfWeekIndex': 0},
   'deliveryAddress': {'addressName': '',
    'zipCode': '06236',
    'baseAddress': '서울특별시 강남구 테헤란로 142 (역삼동, 캐피탈타워)',
    'detailAddress': '아크플레이스 21층',
    'roadNameYn': 'N',
    'primaryYn': 'Y',
    'districtNameText': '서울시 강남구',
    'roadNameText': '테헤란로 142',
    'hasErrorWhileFetchingAddress'

2.  스마트 스토어
- 추세 확인 (시계열)

- 찜/리뷰/빅파워 개수

- 리뷰수 비율
(가장 많은거와 낮은거 비율을 나눠서 범주형으로 전처리)

- 네이버 쇼핑에 __NEXT_DATA__를  json형식으로 가져와 relatedTags 추출

-mallGrade
M44001 : 플래티넘
M44002  : 프리미엄
M44003 : 빅파워
M44004 : 파워
M44005 : 새싹
M44006 : 일반

-1등 스토어 들어가서 best-product.js에 
benefitsView, sellerTags 확인하기

- 상품 심리학 분석

- 구글 바드에서 키워드 검색 
ex ) 스마트스토어에서 물건을 팔고싶은데 최근 유튜브 검색량이 가장 많은 물건 키워드 10개를 리스트업해주세요.

-chatgpt 프롬프트 활용 
ex )  너는 블로그 분야의 전문가입니다. 가습기 판매를 하기위한 자연스러운 리뷰글을 작성해주세요.


-claude AI
여기에서 파일 넣고 질문하기 가능

- 리뷰 ( 10-50-100-1000 )
- 빅파워 개수 (일반 - 파워 - 빅파워 - 플래티넘/프리미엄 )
